# JWST NIRCam row-by-row background subtraction 

Henry Hsieh, updated 2024-05-02

- Performs mitigation of "striped" background structure in NIRCam data by computing row-by-row median values of data files (omitting pixels associated with sources identified by an extended source detection function from median calculations), and subtracting this median value from pixels in each row
- Extended source detection functionality based on https://spacetelescope.github.io/jdat_notebooks/notebooks/NIRCam_photometry/NIRCam_multiband_photometry.html

## Import required packages

In [ ]:
import os,glob
import sys
import time,datetime
import numpy as np
import statistics
from astropy.io import fits
from astropy.convolution import convolve
import astropy.wcs as wcs
import astropy.units as u
from photutils.background import Background2D, MedianBackground
from photutils.segmentation import (detect_sources, deblend_sources, SourceCatalog,
                                    make_2dgaussian_kernel, SegmentationImage)


## Define function to return bandpass-specific parameters

- Zero point data from reference file linked from https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-performance/nircam-absolute-flux-calibration-and-zeropoints#NIRCamAbsoluteFluxCalibrationandZeropoints-NRC_zeropointsZeropoints
  - `"jwst_1126.pmap", delivered Sep. 14, 2023: NRC_ZPs_1126pmap.txt`<br>
    https://jwst-docs.stsci.edu/files/182256933/224166043/1/1695068757137/NRC_ZPs_1126pmap.txt
    

In [ ]:
def get_bandpass_specific_parameters_files(bandpass,base_path):    
    if bandpass == 'F200W':
        pix_area   = 2.23E-14 * u.sr                              #Nominal pixel area in steradians from data file header (PIXAR_SR)
        photmjsr   = 2.055999994277954 * u.MJy/u.sr/(u.DN/u.s)    #Flux density (MJy/steradian) producing 1 cps from data file header (PHOTMJSR)
        updated_zp = 2.056 * u.MJy/u.sr/(u.DN/u.s)                #updated 2024-01-24 from https://jwst-docs.stsci.edu/files/182256933/224166043/1/1695068757137/NRC_ZPs_1126pmap.txt
        zp_corr    = updated_zp / photmjsr
        gain_file  = base_path + 'jwst_nircam_gain_0090.fits'     #Gain reference file name from data file header (R_GAIN)
    if bandpass == 'F277W':
        pix_area   = 9.2373983996251E-14 * u.sr                   #updated 2024-01-24 from jwst_nircam_photom_0153.fits
        photmjsr   = 0.41999998688697815 * u.MJy/u.sr/(u.DN/u.s)  #Flux density (MJy/steradian) producing 1 cps from data file header (PHOTMJSR)
        updated_zp = 0.420 * u.MJy/u.sr/(u.DN/u.s)                #updated 2024-01-24 from https://jwst-docs.stsci.edu/files/182256933/224166043/1/1695068757137/NRC_ZPs_1126pmap.txt
        zp_corr    = updated_zp / photmjsr
        gain_file  = base_path + 'jwst_nircam_gain_0089.fits'     #Gain reference file name from data file header (R_GAIN)
    return pix_area,zp_corr,gain_file


## Define functions to extract image file data

In [ ]:
def extract_image_file_data(infile,pix_area_sr):    
    hdu         = fits.open(infile)
    data_MJySr  = hdu[1].data
    data_MJy    = data_MJySr * pix_area_sr.value
    error_MJySr = hdu[2].data
    error_MJy   = error_MJySr * pix_area_sr.value
    imwcs       = wcs.WCS(hdu[1].header,hdu)
    return data_MJy,error_MJy,imwcs

def show_image_size_FOV(infile,data,imwcs):
    print('Image size for {:s}: '.format(os.path.basename(infile)),end='')
    ny, nx = data.shape
    pixscale = wcs.utils.proj_plane_pixel_scales(imwcs)[0] 
    pixscale *= imwcs.wcs.cunit[0].to('arcsec')
    outline = '%d x %d pixels' % (ny, nx)
    outline += ' = %g" x %g"' % (ny * pixscale, nx * pixscale)
    outline += ' (%.2f" / pixel)' % pixscale
    print(outline)
    return None


## Define function to detect and deblend extended sources

In [ ]:
def detect_sources_and_deblend(data,imwcs,base_path,segm_map_filename):
    # For detection, requiring 5 connected pixels 2-sigma above background

    # Measure background and set detection threshold
    bkg_estimator = MedianBackground()
    bkg = Background2D(data,(50,50),filter_size=(3,3),bkg_estimator=bkg_estimator)
    threshold = bkg.background + (2.*bkg.background_rms)

    # Before detection, smooth image with Gaussian FWHM = 3 pixels
    smooth_kernel = make_2dgaussian_kernel(3.0, size=3)
    convolved_data = convolve(data, smooth_kernel)

    # Detect and deblend
    segm_detect = detect_sources(convolved_data,threshold,npixels=5)
    segm_deblend = deblend_sources(convolved_data,segm_detect,npixels=5,nlevels=32,contrast=0.001)

    # Save segmentation map of detected objects
    segm_hdu = fits.PrimaryHDU(segm_deblend.data.astype(np.uint32),header=imwcs.to_header())
    segm_hdu.writeto(base_path+segm_map_filename,overwrite=True)

    return bkg,segm_deblend
    

## Define function to create masked data file from extended source detection map

In [ ]:
def make_masked_data_file(image_path_data,image_path_segm):
    with fits.open(image_path_data) as hdu_data, fits.open(image_path_segm) as hdu_segm:
        data              = hdu_data[1].data
        data_hdr          = hdu_data[0].header
        segm              = hdu_segm[0].data
        ny_data,nx_data   = data.shape
        data_masked = [[0 for idx in range(nx_data)] for idx in range(ny_data)]
        for idx1 in range(nx_data):
            for idx2 in range(ny_data):
                if segm[idx1][idx2] == 0:
                    data_masked[idx1][idx2] = data[idx1][idx2]
                else:
                    data_masked[idx1][idx2] = 0
        image_path_masked = image_path_data[:-5]+'_masked.fits'
        fits.writeto(image_path_masked,data_masked,data_hdr,overwrite=True,checksum=True)
    return image_path_masked


## Define function to perform row-by-row background subtraction

In [ ]:
def make_rowbgsub_data_file(image_path_data,image_path_masked):
    with fits.open(image_path_data) as hdu_data, fits.open(image_path_masked) as hdu_data_masked:
        data              = hdu_data[1].data
        data_hdr0         = hdu_data[0].header
        data_hdr1         = hdu_data[1].header
        data_hdr          = data_hdr0 + data_hdr1
        data_masked       = hdu_data_masked[0].data
        ny_data,nx_data   = data.shape
        data_rowbgsub     = [[] for idx in range(ny_data)]
        for idx1 in range(ny_data):
            row_data = []
            for idx2 in range(nx_data):
                if data_masked[idx1][idx2] > 0:
                    row_data.append(data[idx1][idx2])
            if row_data != []:
                data_rowbgsub[idx1] = data[idx1] - statistics.median(row_data)
            else:
                data_rowbgsub[idx1] = data[idx1]
        image_path_rowbgsub = image_path_data[:-5]+'_rowbgsub.fits'
        fits.writeto(image_path_rowbgsub,data_rowbgsub,data_hdr,overwrite=True,checksum=True)
    return image_path_rowbgsub


## Define function to perform end-to-end background correction

In [ ]:
def background_subtraction(base_path,bandpass,fits_filename_string):
    print('Performing row-by-row median subtraction for data in {:s}...'.format(base_path))

    os.chdir(base_path)
    pix_area,zp_corr,gain_file = get_bandpass_specific_parameters_files(bandpass,base_path)
    
    for fits_filename in sorted(glob.glob(fits_filename_string)):
        image_path_data = base_path + fits_filename
        
        print('{:s} - Extracting image data from {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),image_path_data))
        data,error,imwcs = extract_image_file_data(image_path_data,pix_area)
        show_image_size_FOV(image_path_data,data,imwcs)
        
        print('{:s} - Generating deblended extended source detection map for {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),image_path_data))
        segm_map_filename = 'detections_segm_' + fits_filename
        bkg,segm_deblend = detect_sources_and_deblend(data,imwcs,base_path,segm_map_filename)
        
        print('{:s} - Performing row-by-row median subtraction for {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),image_path_data))
        image_path_masked   = make_masked_data_file(image_path_data,base_path + segm_map_filename)
        image_path_rowbgsub = make_rowbgsub_data_file(image_path_data,image_path_masked)

    print('Row-by-row median subtraction complete.')
    print('\n{:s} output files:'.format(bandpass))
    for processed_fits_file in sorted(glob.glob(fits_filename_string[:-5]+'_rowbgsub.fits')):
        print(processed_fits_file)
    print('')
    return None


## Perform end-to-end background correction for F200W and F277W observations of 358P/PANSTARRS (Program 4250)

- Set base path (including trailing "/") to location of data files before running

In [ ]:
base_path = ''
background_subtraction(base_path,'F200W','jw04250004001_03101_0000?_nrcb1_cal.fits')
background_subtraction(base_path,'F277W','jw04250004001_03101_0000?_nrcblong_cal.fits')
